In [ ]:
import pandas as pd
import plotly.express as px
import numpy as np
import torch
import torch.nn as nn


# Import data from csv

In [477]:
df = pd.read_csv('merge_data.csv')
df['time'] = df['time'].astype('datetime64[ns]')
df.dtypes

time             datetime64[ns]
currentSpeed              int64
weatherDesc              object
motorcycles               int64
cars                      int64
heavyVehicles             int64
dtype: object

# Add new columns for further analysis

In [478]:
collected_area = 1500

df['total_vehicle'] = df['cars'] + df['motorcycles'] + df['heavyVehicles']
df['Day'] = df['time'].dt.day_name()
df['density'] = (
    (df['cars']) +
    (df['motorcycles'] * 0.5) + 
    (df['heavyVehicles'] * 2)
) / collected_area

df

time  currentSpeed       weatherDesc  motorcycles  cars  \
0   2025-03-12 10:30:03            46  scattered clouds            0    13   
1   2025-03-12 11:00:04            49  scattered clouds            1     9   
2   2025-03-12 11:30:04            45  scattered clouds            1    13   
3   2025-03-12 12:00:04            49  scattered clouds            0     5   
4   2025-03-12 12:30:04            51  scattered clouds            4    11   
..                  ...           ...               ...          ...   ...   
846 2025-04-01 22:01:40            66        few clouds            0     0   
847 2025-04-01 22:31:40            66        few clouds            0     1   
848 2025-04-01 23:01:40            66        few clouds            0     0   
849 2025-04-01 23:31:40            66        light rain            0     3   
850 2025-04-02 00:01:40            66        few clouds            0     0   

     heavyVehicles  total_vehicle        Day   density  
0                0             13  Wednesday  0.008667  
1                0             10  Wednesday  0.006333  
2                0             14  Wednesday  0.009000  
3                0              5  Wednesday  0.003333  
4                0             15  Wednesday  0.008667  
..             ...            ...        ...       ...  
846              0              0    Tuesday  0.000000  
847              0              1    Tuesday  0.000667  
848              0              0    Tuesday  0.000000  
849              0              3    Tuesday  0.002000  
850              0              0  Wednesday  0.000000  

[851 rows x 9 columns]

# EDA

In [479]:
df.describe()

time  currentSpeed  motorcycles        cars  \
count                            851    851.000000   851.000000  851.000000   
mean   2025-03-22 16:14:35.518213888     55.507638     0.846063    6.854289   
min              2025-03-12 10:30:03     22.000000     0.000000    0.000000   
25%       2025-03-17 02:45:23.500000     46.000000     0.000000    0.000000   
50%              2025-03-23 02:30:13     66.000000     0.000000    5.000000   
75%              2025-03-28 13:16:00     66.000000     1.000000   11.000000   
max              2025-04-02 00:01:40     66.000000    13.000000   39.000000   
std                              NaN     11.760087     1.645255    7.639366   

       heavyVehicles  total_vehicle     density  
count     851.000000     851.000000  851.000000  
mean        0.245593       7.945946    0.005179  
min         0.000000       0.000000    0.000000  
25%         0.000000       0.000000    0.000000  
50%         0.000000       6.000000    0.004000  
75%         0.000000      13.000000    0.008333  
max         3.000000      46.000000    0.030000  
std         0.585784       8.836256    0.005730

In [480]:
VEHICLE_FEATURE = ['cars', 'motorcycles',  'heavyVehicles']
FLOW_FEATURE = ['currentSpeed'] 

## Line chart showing relation between number of each type of vehicle and time

In [481]:
# Additional 30 min period column
base_time = pd.to_datetime('00:00:00')
total_minute = ((df['time'].dt.minute + (df['time'].dt.hour * 60)) // 30) * 30 
df['half_hour_interval'] = (base_time + pd.to_timedelta(total_minute, unit='m')).dt.time

In [482]:
px.line(df, x="time", y=VEHICLE_FEATURE)


## Line chart showing relation between number of each type of vehicle and time

In [483]:
px.line(df, x="time", y=FLOW_FEATURE)


## FLow and number of each vehicle on a specific date

In [484]:
def relation_with_time(selected_attr, title='title', xla='x', yla='y'):
    filtered_df = df.loc[df['time'].dt.date == pd.Timestamp('2025-03-30').date()]
    fig = px.line(filtered_df, x="time", y=selected_attr)
    fig.update_layout(
        title=title,
        xaxis_title=xla,
        yaxis_title=yla
    )
    fig.show()
    

relation_with_time(VEHICLE_FEATURE, 'Vehicle count on specific day', 'Time', 'Vehicle count')
relation_with_time(FLOW_FEATURE, 'Traffic flow on specific day', 'Time', 'Traffic flow')

df

time  currentSpeed       weatherDesc  motorcycles  cars  \
0   2025-03-12 10:30:03            46  scattered clouds            0    13   
1   2025-03-12 11:00:04            49  scattered clouds            1     9   
2   2025-03-12 11:30:04            45  scattered clouds            1    13   
3   2025-03-12 12:00:04            49  scattered clouds            0     5   
4   2025-03-12 12:30:04            51  scattered clouds            4    11   
..                  ...           ...               ...          ...   ...   
846 2025-04-01 22:01:40            66        few clouds            0     0   
847 2025-04-01 22:31:40            66        few clouds            0     1   
848 2025-04-01 23:01:40            66        few clouds            0     0   
849 2025-04-01 23:31:40            66        light rain            0     3   
850 2025-04-02 00:01:40            66        few clouds            0     0   

     heavyVehicles  total_vehicle        Day   density half_hour_interval  
0                0             13  Wednesday  0.008667           10:30:00  
1                0             10  Wednesday  0.006333           11:00:00  
2                0             14  Wednesday  0.009000           11:30:00  
3                0              5  Wednesday  0.003333           12:00:00  
4                0             15  Wednesday  0.008667           12:30:00  
..             ...            ...        ...       ...                ...  
846              0              0    Tuesday  0.000000           22:00:00  
847              0              1    Tuesday  0.000667           22:30:00  
848              0              0    Tuesday  0.000000           23:00:00  
849              0              3    Tuesday  0.002000           23:30:00  
850              0              0  Wednesday  0.000000           00:00:00  

[851 rows x 10 columns]

## Average number of each type of vehicle and traffic flow in each day of week

In [485]:
def avg_on_day_of_week(selected_attr, title='title', xla='x', yla='y'):
    
    color_map = {
        "Monday": "yellow",
        "Tuesday": "pink",
        "Wednesday": "green",
        "Thursday": "orange",
        "Friday": "blue",
        "Saturday": "purple",
        "Sunday": "red"
    }

    # Extract day names
    df['Day'] = df['time'].dt.day_name()

    grouped = df.groupby(['Day', 'half_hour_interval'])[selected_attr].mean().reset_index().sort_values('half_hour_interval', ascending=False)
    
    fig = px.line(
        grouped, 
        y=selected_attr, 
        x='half_hour_interval',
        color='Day',
        color_discrete_map=color_map,
    )
    fig.update_layout(
        xaxis=dict(title=xla, autorange="reversed"),
        title=title,
        yaxis_title=yla
    )
    fig.show()
    
avg_on_day_of_week(['total_vehicle'], 'Average vehicle count on each day of week every 30 minute', '30 minute interval', 'Vehicle count')
avg_on_day_of_week(FLOW_FEATURE, 'Average traffic flow on each day of week every 30 minute', '30 minute interval', 'traffic flow')




## Average number of each type of vehicle in each hour of day

In [486]:
def avg_on_hour(selected_attr, title='title', xla='x', yla='y'):
    byhour = df.groupby(df['half_hour_interval'])[selected_attr].mean()
    fig = px.line(byhour, y=selected_attr, x=byhour.index)
    fig.update_xaxes(
        tickmode='linear'
    )
    fig.update_layout(
        title=title,
        xaxis_title=xla,
        yaxis_title=yla
    )
    fig.show()

avg_on_hour(VEHICLE_FEATURE, 'Average vehicle type count every 30 minute of day', '30 minute interval', 'Vehicle count')
avg_on_hour(FLOW_FEATURE, 'Average traffic flow 30 minute of day', '30 minute interval', 'traffic flow')

## Average number of each vehicle type in each weather condition. 

In [487]:
def avg_on_weather_conditiopn(selected_attr, title='title', xla='x', yla='y'):
    notime = df.drop(columns=['time'])
    weather = notime.groupby('weatherDesc')[selected_attr].mean()

    fig = px.bar(weather, y=selected_attr, x=weather.index, barmode='group')
    fig.update_layout(
        title=title,
        xaxis_title=xla,
        yaxis_title=yla
    )
    fig.show()

avg_on_weather_conditiopn(VEHICLE_FEATURE, 'Average vehicle type count on each weather condition', 'weather condition', 'Vehicle count')
avg_on_weather_conditiopn(FLOW_FEATURE, 'Average traffic flow on each weather condition', 'weather condition', 'traffic flow')

# Data preprocessing

## Convert hour to float

In [489]:
train_df = df.copy()

train_df['hour'] = (((train_df['time'].dt.hour * 60) + train_df['time'].dt.minute) // 30) / 2

train_df['weatherDesc'].unique()

array(['scattered clouds', 'few clouds', 'overcast clouds', 'light rain',
       'moderate rain', 'broken clouds', 'clear sky',
       'heavy intensity rain', 'haze', 'mist'], dtype=object)

## Convert day to is_weekend

In [490]:
from sklearn.preprocessing import LabelEncoder 

def categorize_weather(wether_desc):
    
    if wether_desc in ['clear sky', 'few clouds', 'scattered clouds']:
        return 0
    
    if wether_desc in ['broken clouds', 'overcast clouds']:
        return 1
    
    if wether_desc in ['haze', 'mist']:
        return 2
    
    if wether_desc in ['light rain', 'moderate rain', 'heavy intensity rain']:
        return 3

train_df['ec_weather'] = train_df['weatherDesc'].map(categorize_weather)

train_df['is_weekend'] = train_df['Day'].map(
    lambda day: day in ['Saturday', 'Sunday']
).astype('int8')
train_df

time  currentSpeed       weatherDesc  motorcycles  cars  \
0   2025-03-12 10:30:03            46  scattered clouds            0    13   
1   2025-03-12 11:00:04            49  scattered clouds            1     9   
2   2025-03-12 11:30:04            45  scattered clouds            1    13   
3   2025-03-12 12:00:04            49  scattered clouds            0     5   
4   2025-03-12 12:30:04            51  scattered clouds            4    11   
..                  ...           ...               ...          ...   ...   
846 2025-04-01 22:01:40            66        few clouds            0     0   
847 2025-04-01 22:31:40            66        few clouds            0     1   
848 2025-04-01 23:01:40            66        few clouds            0     0   
849 2025-04-01 23:31:40            66        light rain            0     3   
850 2025-04-02 00:01:40            66        few clouds            0     0   

     heavyVehicles  total_vehicle        Day   density half_hour_interval  \
0                0             13  Wednesday  0.008667           10:30:00   
1                0             10  Wednesday  0.006333           11:00:00   
2                0             14  Wednesday  0.009000           11:30:00   
3                0              5  Wednesday  0.003333           12:00:00   
4                0             15  Wednesday  0.008667           12:30:00   
..             ...            ...        ...       ...                ...   
846              0              0    Tuesday  0.000000           22:00:00   
847              0              1    Tuesday  0.000667           22:30:00   
848              0              0    Tuesday  0.000000           23:00:00   
849              0              3    Tuesday  0.002000           23:30:00   
850              0              0  Wednesday  0.000000           00:00:00   

     hour  ec_weather  is_weekend  
0    10.5           0           0  
1    11.0           0           0  
2    11.5           0           0  
3    12.0           0           0  
4    12.5           0           0  
..    ...         ...         ...  
846  22.0           0           0  
847  22.5           0           0  
848  23.0           0           0  
849  23.5           3           0  
850   0.0           0           0  

[851 rows x 13 columns]

## Creating sequence function

In [491]:
def create_seq(data, seq_length):
    seq = []
    target = []
    for i in range(len(data) - seq_length):
        seq.append(data[i: i + seq_length, :-1])
        target.append(data[i: i + seq_length, -1])
    return torch.FloatTensor(seq), torch.FloatTensor(target)

## Create model class

In [ ]:
class TrafficGRU(nn.Module):
    def __init__(self, weather_cat, hidden_size=64, num_layers=2):
        super().__init__()
        
        input_size = len(weather_cat) + 2 # Number of weather condition + sin/cos hour + is_weekend
        self.weather_cat_count = len(weather_cat)
        self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, 1) 
        self.ln = nn.LayerNorm(hidden_size) 
        self.relu = nn.ReLU(inplace=True)   

    def preprocess_param(self, hour, weather):

        sin_hour = torch.sin(2 * np.pi * hour / 24).unsqueeze(0)
        sin_cos = torch.cos(2 * np.pi * hour / 24).unsqueeze(0)
        
        int_weather = int(weather.item())
        weather_one_hot = torch.nn.functional.one_hot(
            torch.tensor(int_weather),
            num_classes=self.weather_cat_count
        ).float()
        
        return torch.cat([sin_cos, sin_hour, weather_one_hot])
        
    def forward(self, x):
        
        processed = []
        for i in x:
            
            batch = []
            
            for j in i:
                
                hour, weather = j[0], j[1]
                feature = self.preprocess_param(hour, weather)
                batch.append(feature)
            
            processed.append(torch.stack(batch))
        
        gru_input = torch.stack(processed)
        
        h0 = torch.zeros(self.gru.num_layers, x.size(0), self.gru.hidden_size)
        out, _ = self.gru(gru_input , h0)
        out = self.ln(out)
        out = self.fc(out[:, -1, :])
        out = self.relu(out)
        return out

# Train number of car prediction model

## Set up model

In [493]:
# weather_cat = list(train_df['ec_weather'].unique())
# y_min = df['cars'].min()
# y_max = df['cars'].max()

# # Set up hyper parameter
# hidden_size = 64
# num_layers = 2

# train_df.sort_values('time', inplace=True)
# car_train, car_test = np.split(train_df[['hour', 'ec_weather', 'cars']], [int(len(train_df) * 0.8)])
# X, y = create_seq(car_train.values, 2)

# model = TrafficGRU(weather_cat, y_min, y_max, hidden_size, num_layers)
# criterion = nn.MSELoss()
# optimizer = torch.optim.Adam(model.parameters())

# car_train, car_test


## Training loop

In [494]:
# from torch.utils.data import DataLoader, TensorDataset

# dataset = TensorDataset(X, y)
# dataloader = DataLoader(dataset, batch_size=24)

# for e in range(50): 
#     for batch_x, batch_y in dataloader:
#         outputs = model(batch_x)
#         loss = criterion(outputs, batch_y)
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()
#     print(f'Epoch {e+1}, Loss: {loss.item():.4f}')

In [495]:
# input = torch.tensor([
#     [
#         [0, 0, 4],
#         [0.5, 0, 4],
#     ], 
#     ])
# with torch.no_grad():
#     prediction = model(input)
#     print(prediction)
    
# input.shape

In [496]:
# model.eval()
# scripted_model = torch.jit.script(model)  # or torch.jit.trace(model, input_example)
# scripted_model.save("carmodel_less_feature.pt")

In [497]:
# X_test, y_test = create_seq(car_test.values, 2)
# test_criterion = nn.MSELoss()

# with torch.no_grad():
#     predictions = model(X_test)
#     test_loss = test_criterion(predictions, y_test)
#     print(f'Test Loss: {test_loss.item():.4f}')

# Create and training model function

In [498]:
from torch.utils.data import DataLoader, TensorDataset

def create_and_train_model(target_feature, epoch, lr=0.001):
    weather_cat = list(train_df['ec_weather'].unique())

    # Set up hyper parameter
    hidden_size = 64
    num_layers = 2

    train_df.sort_values('time', inplace=True)
    train_set, car_test = np.split(train_df[['hour', 'ec_weather'] + [target_feature]], [int(len(train_df) * 0.8)])
    X, y = create_seq(train_set.values, 2)

    model = TrafficGRU(weather_cat, hidden_size, num_layers)
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    dataset = TensorDataset(X, y)
    dataloader = DataLoader(dataset, batch_size=24)
    
    prev_lost = -1

    for e in range(epoch): 
        for batch_x, batch_y in dataloader:
            outputs = model(batch_x)
            loss = criterion(outputs, batch_y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        print(f'Epoch {e+1}, Loss: {loss.item():.4f}')    
    
    # while(True):
    #     for batch_x, batch_y in dataloader:
    #         outputs = model(batch_x)
    #         loss = criterion(outputs, batch_y)
    #         optimizer.zero_grad()
    #         loss.backward()
    #         optimizer.step()
    #     print(f'Epoch {e+1}, Loss: {loss.item():.4f}')    
    #     if (int(loss) == prev_lost):
    #         break;
    #     prev_lost = int(loss)
        
    X_test, y_test = create_seq(car_test.values, 2)
    test_criterion = nn.MSELoss()

    with torch.no_grad():
        predictions = model(X_test)
        test_loss = test_criterion(predictions, y_test)
    
    return  model, test_loss.item()    

# Train model of each target feature

In [499]:
car_model, car_per = create_and_train_model('cars', 50)
print("Number of car prediction model loss: ", car_per)

/Users/Ohm/Desktop/ske/daq/construct_model/.env/lib/python3.13/site-packages/numpy/_core/fromnumeric.py:57: FutureWarning:

'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.

/Users/Ohm/Desktop/ske/daq/construct_model/.env/lib/python3.13/site-packages/torch/nn/modules/loss.py:610: UserWarning:

Using a target size (torch.Size([24, 2])) that is different to the input size (torch.Size([24, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.

/Users/Ohm/Desktop/ske/daq/construct_model/.env/lib/python3.13/site-packages/torch/nn/modules/loss.py:610: UserWarning:

Using a target size (torch.Size([6, 2])) that is different to the input size (torch.Size([6, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.



Epoch 1, Loss: 70.7442
Epoch 2, Loss: 59.8207
Epoch 3, Loss: 48.2697
Epoch 4, Loss: 42.4941
Epoch 5, Loss: 39.6615
Epoch 6, Loss: 38.8192
Epoch 7, Loss: 38.6714
Epoch 8, Loss: 39.1179
Epoch 9, Loss: 38.9019
Epoch 10, Loss: 38.1165
Epoch 11, Loss: 38.3176
Epoch 12, Loss: 38.5709
Epoch 13, Loss: 38.9736
Epoch 14, Loss: 39.2387
Epoch 15, Loss: 39.4147
Epoch 16, Loss: 39.4889
Epoch 17, Loss: 39.5253
Epoch 18, Loss: 39.5409
Epoch 19, Loss: 39.5477
Epoch 20, Loss: 39.5542
Epoch 21, Loss: 39.5687
Epoch 22, Loss: 39.5909
Epoch 23, Loss: 39.6171
Epoch 24, Loss: 39.6441
Epoch 25, Loss: 39.6683
Epoch 26, Loss: 39.6881
Epoch 27, Loss: 39.7027
Epoch 28, Loss: 39.7123
Epoch 29, Loss: 39.7174
Epoch 30, Loss: 39.7222
Epoch 31, Loss: 39.7206
Epoch 32, Loss: 39.7175
Epoch 33, Loss: 39.7115
Epoch 34, Loss: 39.7083
Epoch 35, Loss: 39.7062
Epoch 36, Loss: 39.7027
Epoch 37, Loss: 39.6991
Epoch 38, Loss: 39.6954
Epoch 39, Loss: 39.6918
Epoch 40, Loss: 39.6882
Epoch 41, Loss: 39.6847
Epoch 42, Loss: 39.6812
E

/Users/Ohm/Desktop/ske/daq/construct_model/.env/lib/python3.13/site-packages/torch/nn/modules/loss.py:610: UserWarning:

Using a target size (torch.Size([169, 2])) that is different to the input size (torch.Size([169, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.



In [500]:
mot_model, mot_per = create_and_train_model('motorcycles', 50)
print("Number of motorcycles prediction model loss: ", mot_per)

/Users/Ohm/Desktop/ske/daq/construct_model/.env/lib/python3.13/site-packages/numpy/_core/fromnumeric.py:57: FutureWarning:

'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.

/Users/Ohm/Desktop/ske/daq/construct_model/.env/lib/python3.13/site-packages/torch/nn/modules/loss.py:610: UserWarning:

Using a target size (torch.Size([24, 2])) that is different to the input size (torch.Size([24, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.

/Users/Ohm/Desktop/ske/daq/construct_model/.env/lib/python3.13/site-packages/torch/nn/modules/loss.py:610: UserWarning:

Using a target size (torch.Size([6, 2])) that is different to the input size (torch.Size([6, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.



Epoch 1, Loss: 10.6600
Epoch 2, Loss: 10.5933
Epoch 3, Loss: 10.6194
Epoch 4, Loss: 10.5726
Epoch 5, Loss: 10.3984
Epoch 6, Loss: 10.1647
Epoch 7, Loss: 9.9419
Epoch 8, Loss: 9.7372
Epoch 9, Loss: 9.6122
Epoch 10, Loss: 9.3034
Epoch 11, Loss: 9.0139
Epoch 12, Loss: 8.8860
Epoch 13, Loss: 8.7858
Epoch 14, Loss: 8.7166
Epoch 15, Loss: 8.6870
Epoch 16, Loss: 8.6033
Epoch 17, Loss: 8.5762
Epoch 18, Loss: 8.6425
Epoch 19, Loss: 8.6275
Epoch 20, Loss: 8.5958
Epoch 21, Loss: 8.5864
Epoch 22, Loss: 8.5972
Epoch 23, Loss: 8.5862
Epoch 24, Loss: 8.5863
Epoch 25, Loss: 8.5675
Epoch 26, Loss: 8.5806
Epoch 27, Loss: 8.5491
Epoch 28, Loss: 8.5725
Epoch 29, Loss: 8.5318
Epoch 30, Loss: 8.5539
Epoch 31, Loss: 8.5188
Epoch 32, Loss: 8.5279
Epoch 33, Loss: 8.5036
Epoch 34, Loss: 8.5053
Epoch 35, Loss: 8.4854
Epoch 36, Loss: 8.4842
Epoch 37, Loss: 8.4665
Epoch 38, Loss: 8.4634
Epoch 39, Loss: 8.4465
Epoch 40, Loss: 8.4428
Epoch 41, Loss: 8.4258
Epoch 42, Loss: 8.4221
Epoch 43, Loss: 8.4044
Epoch 44, Loss

/Users/Ohm/Desktop/ske/daq/construct_model/.env/lib/python3.13/site-packages/torch/nn/modules/loss.py:610: UserWarning:

Using a target size (torch.Size([169, 2])) that is different to the input size (torch.Size([169, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.



In [501]:
heavy_model, heavy_per = create_and_train_model('heavyVehicles', 50)
print("Number of heavy vehicles prediction model loss: ", heavy_per)

Epoch 1, Loss: 0.2500


/Users/Ohm/Desktop/ske/daq/construct_model/.env/lib/python3.13/site-packages/numpy/_core/fromnumeric.py:57: FutureWarning:

'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.

/Users/Ohm/Desktop/ske/daq/construct_model/.env/lib/python3.13/site-packages/torch/nn/modules/loss.py:610: UserWarning:

Using a target size (torch.Size([24, 2])) that is different to the input size (torch.Size([24, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.

/Users/Ohm/Desktop/ske/daq/construct_model/.env/lib/python3.13/site-packages/torch/nn/modules/loss.py:610: UserWarning:

Using a target size (torch.Size([6, 2])) that is different to the input size (torch.Size([6, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.



Epoch 2, Loss: 0.2500
Epoch 3, Loss: 0.2500
Epoch 4, Loss: 0.2500
Epoch 5, Loss: 0.2500
Epoch 6, Loss: 0.2500
Epoch 7, Loss: 0.2500
Epoch 8, Loss: 0.2500
Epoch 9, Loss: 0.2500
Epoch 10, Loss: 0.2500
Epoch 11, Loss: 0.2500
Epoch 12, Loss: 0.2500
Epoch 13, Loss: 0.2500
Epoch 14, Loss: 0.2500
Epoch 15, Loss: 0.2500
Epoch 16, Loss: 0.2500
Epoch 17, Loss: 0.2500
Epoch 18, Loss: 0.2500
Epoch 19, Loss: 0.2500
Epoch 20, Loss: 0.2500
Epoch 21, Loss: 0.2500
Epoch 22, Loss: 0.2500
Epoch 23, Loss: 0.2500
Epoch 24, Loss: 0.2500
Epoch 25, Loss: 0.2500
Epoch 26, Loss: 0.2500
Epoch 27, Loss: 0.2500
Epoch 28, Loss: 0.2500
Epoch 29, Loss: 0.2500
Epoch 30, Loss: 0.2500
Epoch 31, Loss: 0.2500
Epoch 32, Loss: 0.2500
Epoch 33, Loss: 0.2500
Epoch 34, Loss: 0.2500
Epoch 35, Loss: 0.2500
Epoch 36, Loss: 0.2500
Epoch 37, Loss: 0.2500
Epoch 38, Loss: 0.2500
Epoch 39, Loss: 0.2500
Epoch 40, Loss: 0.2500
Epoch 41, Loss: 0.2500
Epoch 42, Loss: 0.2500
Epoch 43, Loss: 0.2500
Epoch 44, Loss: 0.2500
Epoch 45, Loss: 0.2

/Users/Ohm/Desktop/ske/daq/construct_model/.env/lib/python3.13/site-packages/torch/nn/modules/loss.py:610: UserWarning:

Using a target size (torch.Size([169, 2])) that is different to the input size (torch.Size([169, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.



In [502]:
speed_model, speed_per = create_and_train_model('currentSpeed', 200, 0.01)
print("Traffic flow prediction model loss: ", speed_per)

Epoch 1, Loss: 2966.2500


/Users/Ohm/Desktop/ske/daq/construct_model/.env/lib/python3.13/site-packages/numpy/_core/fromnumeric.py:57: FutureWarning:

'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.

/Users/Ohm/Desktop/ske/daq/construct_model/.env/lib/python3.13/site-packages/torch/nn/modules/loss.py:610: UserWarning:

Using a target size (torch.Size([24, 2])) that is different to the input size (torch.Size([24, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.

/Users/Ohm/Desktop/ske/daq/construct_model/.env/lib/python3.13/site-packages/torch/nn/modules/loss.py:610: UserWarning:

Using a target size (torch.Size([6, 2])) that is different to the input size (torch.Size([6, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.



Epoch 2, Loss: 2966.2500
Epoch 3, Loss: 2966.2500
Epoch 4, Loss: 2966.2500
Epoch 5, Loss: 2966.2500
Epoch 6, Loss: 2966.2500
Epoch 7, Loss: 2966.2500
Epoch 8, Loss: 2966.2500
Epoch 9, Loss: 2966.2500
Epoch 10, Loss: 2966.2500
Epoch 11, Loss: 2966.2500
Epoch 12, Loss: 2966.2500
Epoch 13, Loss: 2966.2500
Epoch 14, Loss: 2966.2500
Epoch 15, Loss: 2966.2500
Epoch 16, Loss: 2966.2500
Epoch 17, Loss: 2966.2500
Epoch 18, Loss: 2966.2500
Epoch 19, Loss: 2966.2500
Epoch 20, Loss: 2966.2500
Epoch 21, Loss: 2966.2500
Epoch 22, Loss: 2966.2500
Epoch 23, Loss: 2966.2500
Epoch 24, Loss: 2966.2500
Epoch 25, Loss: 2966.2500
Epoch 26, Loss: 2966.2500
Epoch 27, Loss: 2966.2500
Epoch 28, Loss: 2966.2500
Epoch 29, Loss: 2966.2500
Epoch 30, Loss: 2966.2500
Epoch 31, Loss: 2966.2500
Epoch 32, Loss: 2966.2500
Epoch 33, Loss: 2966.2500
Epoch 34, Loss: 2966.2500
Epoch 35, Loss: 2966.2500
Epoch 36, Loss: 2966.2500
Epoch 37, Loss: 2966.2500
Epoch 38, Loss: 2966.2500
Epoch 39, Loss: 2966.2500
Epoch 40, Loss: 2966

/Users/Ohm/Desktop/ske/daq/construct_model/.env/lib/python3.13/site-packages/torch/nn/modules/loss.py:610: UserWarning:

Using a target size (torch.Size([169, 2])) that is different to the input size (torch.Size([169, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.



In [503]:
total_model, total_per = create_and_train_model('total_vehicle', 100)
print("Total number of vehicles prediction model loss: ", total_per)

Epoch 1, Loss: 145.2684
Epoch 2, Loss: 125.2265


/Users/Ohm/Desktop/ske/daq/construct_model/.env/lib/python3.13/site-packages/numpy/_core/fromnumeric.py:57: FutureWarning:

'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.

/Users/Ohm/Desktop/ske/daq/construct_model/.env/lib/python3.13/site-packages/torch/nn/modules/loss.py:610: UserWarning:

Using a target size (torch.Size([24, 2])) that is different to the input size (torch.Size([24, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.

/Users/Ohm/Desktop/ske/daq/construct_model/.env/lib/python3.13/site-packages/torch/nn/modules/loss.py:610: UserWarning:

Using a target size (torch.Size([6, 2])) that is different to the input size (torch.Size([6, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.



Epoch 3, Loss: 107.2409
Epoch 4, Loss: 96.9732
Epoch 5, Loss: 90.9823
Epoch 6, Loss: 88.1927
Epoch 7, Loss: 86.5625
Epoch 8, Loss: 85.7464
Epoch 9, Loss: 85.3924
Epoch 10, Loss: 85.1596
Epoch 11, Loss: 85.0429
Epoch 12, Loss: 84.4280
Epoch 13, Loss: 84.5046
Epoch 14, Loss: 84.8675
Epoch 15, Loss: 85.4092
Epoch 16, Loss: 85.8778
Epoch 17, Loss: 86.2121
Epoch 18, Loss: 86.4275
Epoch 19, Loss: 86.5641
Epoch 20, Loss: 86.6384
Epoch 21, Loss: 86.6647
Epoch 22, Loss: 86.6552
Epoch 23, Loss: 86.6191
Epoch 24, Loss: 86.5646
Epoch 25, Loss: 86.4951
Epoch 26, Loss: 86.4228
Epoch 27, Loss: 86.3467
Epoch 28, Loss: 86.2722
Epoch 29, Loss: 86.1969
Epoch 30, Loss: 86.1229
Epoch 31, Loss: 86.0522
Epoch 32, Loss: 85.9850
Epoch 33, Loss: 85.9214
Epoch 34, Loss: 85.8637
Epoch 35, Loss: 85.8079
Epoch 36, Loss: 85.7551
Epoch 37, Loss: 85.7053
Epoch 38, Loss: 85.6580
Epoch 39, Loss: 85.6136
Epoch 40, Loss: 85.5714
Epoch 41, Loss: 85.5317
Epoch 42, Loss: 85.4938
Epoch 43, Loss: 85.4582
Epoch 44, Loss: 85.424

/Users/Ohm/Desktop/ske/daq/construct_model/.env/lib/python3.13/site-packages/torch/nn/modules/loss.py:610: UserWarning:

Using a target size (torch.Size([169, 2])) that is different to the input size (torch.Size([169, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.



In [504]:
dense_model, dense_per = create_and_train_model('density', 100)
print("Vehicle density prediction model loss: ", dense_per)

Epoch 1, Loss: 0.0001


/Users/Ohm/Desktop/ske/daq/construct_model/.env/lib/python3.13/site-packages/numpy/_core/fromnumeric.py:57: FutureWarning:

'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.

/Users/Ohm/Desktop/ske/daq/construct_model/.env/lib/python3.13/site-packages/torch/nn/modules/loss.py:610: UserWarning:

Using a target size (torch.Size([24, 2])) that is different to the input size (torch.Size([24, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.

/Users/Ohm/Desktop/ske/daq/construct_model/.env/lib/python3.13/site-packages/torch/nn/modules/loss.py:610: UserWarning:

Using a target size (torch.Size([6, 2])) that is different to the input size (torch.Size([6, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.



Epoch 2, Loss: 0.0001
Epoch 3, Loss: 0.0001
Epoch 4, Loss: 0.0001
Epoch 5, Loss: 0.0001
Epoch 6, Loss: 0.0001
Epoch 7, Loss: 0.0001
Epoch 8, Loss: 0.0001
Epoch 9, Loss: 0.0001
Epoch 10, Loss: 0.0001
Epoch 11, Loss: 0.0001
Epoch 12, Loss: 0.0001
Epoch 13, Loss: 0.0001
Epoch 14, Loss: 0.0001
Epoch 15, Loss: 0.0001
Epoch 16, Loss: 0.0001
Epoch 17, Loss: 0.0001
Epoch 18, Loss: 0.0001
Epoch 19, Loss: 0.0001
Epoch 20, Loss: 0.0001
Epoch 21, Loss: 0.0001
Epoch 22, Loss: 0.0001
Epoch 23, Loss: 0.0001
Epoch 24, Loss: 0.0001
Epoch 25, Loss: 0.0001
Epoch 26, Loss: 0.0001
Epoch 27, Loss: 0.0001
Epoch 28, Loss: 0.0001
Epoch 29, Loss: 0.0001
Epoch 30, Loss: 0.0001
Epoch 31, Loss: 0.0001
Epoch 32, Loss: 0.0001
Epoch 33, Loss: 0.0001
Epoch 34, Loss: 0.0001
Epoch 35, Loss: 0.0001
Epoch 36, Loss: 0.0001
Epoch 37, Loss: 0.0001
Epoch 38, Loss: 0.0001
Epoch 39, Loss: 0.0001
Epoch 40, Loss: 0.0001
Epoch 41, Loss: 0.0001
Epoch 42, Loss: 0.0001
Epoch 43, Loss: 0.0001
Epoch 44, Loss: 0.0001
Epoch 45, Loss: 0.0

/Users/Ohm/Desktop/ske/daq/construct_model/.env/lib/python3.13/site-packages/torch/nn/modules/loss.py:610: UserWarning:

Using a target size (torch.Size([169, 2])) that is different to the input size (torch.Size([169, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.



# Prediction model performance (MSE)

In [505]:
from math import sqrt

print(f"""
Number of car prediction model: {car_per:.4}
Number of motorcycle prediction model: {mot_per:.4}
Number of heavy vehicle prediction model: {heavy_per:.4}
Total number vehicle prediction model: {total_per:.4}
Traffic flow prediction model: {speed_per:.4}
Density prediction model: {dense_per:.4}
""")

print(f"""
Number of car prediction model: {sqrt(car_per):.4}
Number of motorcycle prediction model: {sqrt(mot_per):.4}
Number of heavy vehicle prediction model: {sqrt(heavy_per):.4}
Total number vehicle prediction model: {sqrt(total_per):.4}
Traffic flow prediction model: {sqrt(speed_per):.4}
Density prediction model: {sqrt(dense_per):.4}
""")


Number of car prediction model: 13.95
Number of motorcycle prediction model: 2.175
Number of heavy vehicle prediction model: 0.497
Total number vehicle prediction model: 17.43
Traffic flow prediction model: 3.183e+03
Density prediction model: 5.219e-05


Number of car prediction model: 3.734
Number of motorcycle prediction model: 1.475
Number of heavy vehicle prediction model: 0.705
Total number vehicle prediction model: 4.175
Traffic flow prediction model: 56.42
Density prediction model: 0.007224



In [507]:
import os

def export_model(model, model_name): 
    model.eval()
    scripted_model = torch.jit.script(model)  # or torch.jit.trace(model, input_example)
    
    if (not os.path.isdir('model')):
        os.mkdir('model')
    scripted_model.save(f"model/{model_name}.pt")
    
export_model(car_model, "car_num_model")
export_model(mot_model, "motorcycle_vehicle_num_model")
export_model(heavy_model, "heavy_vehicle_num_model")
export_model(total_model, "total_vehicle_num_model")
export_model(speed_model, "traffic_flow_model")